In [1]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama


In [2]:
model = ChatOllama(
    model="scb10x/llama3.2-typhoon2-t1-3b-research-preview"
)


C:\Users\User\AppData\Local\Temp\ipykernel_11220\747115600.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [3]:
messages = [HumanMessage(content="อธิบายกลยุทธ์ปิดฟ้าข้ามทะเล แบบเข้าใจง่าย")]

In [4]:
response = model.invoke(messages)

In [5]:
print(response.content)

<thoughts>

<plan>
<step order="1">ทำความเข้าใจคำว่า "การปิดล้อมทางอากาศ" และบริบท</step>
<step order="2">อธิบายกลยุทธ์โดยใช้ภาษาที่เข้าใจง่ายและเปรียบเทียบกับสถานการณ์ที่คุ้นเคย</step>
<step order="3">สรุปแนวคิดหลักให้ชัดเจน</step>
<step order="4">ตรวจสอบความชัดเจนและความเหมาะสมของคำอธิบาย</step>
</plan>

<step>
<title>ทำความเข้าใจคำว่า "การปิดล้อมทางอากาศ"</title>
<scratch_pad>
"การปิดล้อมทางอากาศ" หมายถึงกลยุทธ์ทางการทหารที่ใช้เครื่องบินเพื่อโจมตีหรือปกป้องพื้นที่ โดยมักเกี่ยวข้องกับการควบคุมอากาศเหนือพื้นที่เป้าหมาย
</scratch_pad>
<summary>ฉันกำลังทำความเข้าใจคำว่า "การปิดล้อมทางอากาศ" เพื่อให้สามารถอธิบายกลยุทธ์นี้ได้อย่างถูกต้องและชัดเจน</summary>
<next_step>อธิบายกลยุทธ์โดยใช้ภาษาที่เข้าใจง่ายและเปรียบเทียบกับสถานการณ์ที่คุ้นเคย</next_step>
</step>

<step>
<title>อธิบายกลยุทธ์โดยใช้ภาษาที่เข้าใจง่าย</title>
<scratch_pad>
1. **การปิดล้อมทางอากาศ** คือการใช้เครื่องบินเพื่อปกป้องหรือโจมตีเป้าหมาย
2. **เปรียบเทียบ**: หากคุณอยู่ในสวนสาธารณะและเห็นคนอีกกลุ่มหนึ่งกำลังเล่นฟุตบอล คุณจะส

In [6]:
import ray
ray.init(ignore_reinit_error=True, include_dashboard=False, num_cpus=4)


Exception: The current node timed out during startup. This could happen because some of the Ray processes failed to startup.

In [ ]:
@ray.remote
class OllamaWorker:
    def __init__(self, model_name: str = "scb10x/llama3.2-typhoon2-t1-3b-research-preview"):
        self.model = ChatOllama(model=model_name)

    def generate(self, prompt: str) -> str:
        # Wrap prompt into a HumanMessage
        message = HumanMessage(content=prompt)
        # Invoke the model
        response = self.model.invoke([message])
        return response.content

In [ ]:
if __name__ == "__main__":
    # List of prompts to process in parallel
    prompts = [
        "อธิบายกลยุทธ์ปิดฟ้าข้ามทะเล แบบเข้าใจง่าย",
    ]

    # Launch a worker for each prompt
    workers = [OllamaWorker.remote() for _ in prompts]
    # Kick off parallel generation tasks
    futures = [worker.generate.remote(prompt) for worker, prompt in zip(workers, prompts)]

    # Gather results
    results = ray.get(futures)

    # Display outputs
    for i, output in enumerate(results, 1):
        print(f"--- Prompt {i} ---")
        print(prompts[i-1])
        print(output)
        print()
